In [1]:
# assistant_abc.py

from abc import ABC, abstractmethod
from typing import List, Tuple

from openai.types.beta.threads.run import Run
from openai.types.beta.threads import ThreadMessage
from openai.types.beta.thread import Thread
from openai.pagination import SyncCursorPage


class OpenAIAssistantBase(ABC):
    """Base Assistant."""

    name: str
    description: str
    model: str
    instructions: str

    @abstractmethod
    def create_assistant(
        self, 
        name: str, 
        description: str, 
        instructions: str
    ) -> str:
        """Creates an assistant with the provided name, description, and instructions."""

    @abstractmethod
    def submit_message(
        self, 
        assistant_id: str, 
        thread_id: str, 
        user_message: str
    ) -> Run:
        """Submit a message to the provided thread."""

    @abstractmethod
    def get_response(
        self, 
        thread
    ) -> SyncCursorPage[ThreadMessage]:
        """Get the response from the provided thread."""

    @abstractmethod
    def create_thread_and_run(
        self, 
        assistant_id: str, 
        user_input: str
    ) -> Tuple[Thread, Run]:
        """Create a thread and submit a message to it."""

    @abstractmethod
    def wait_on_run(
        self, 
        run: Run, 
        thread: Thread
    ) -> Run:
        """Wait for the run to complete."""

    @abstractmethod
    def display_message(
        self, 
        messages: List[ThreadMessage]
    ) -> None:
        """Display the message from the Thread of messages."""

In [2]:
# config_model.py

import yaml

class SKAssistantConfigModel:
    def __init__(self, name: str, description: str, instructions: str):
        self.name = name
        self.description = description
        self.instructions = instructions

    @classmethod
    def load_from_definition_string(cls, string: str) -> "SKAssistantConfigModel":
        yaml_data = yaml.safe_load(string)
        return cls(
            name=yaml_data['name'],
            description=yaml_data['description'],
            instructions=yaml_data['instructions']
        )

    @classmethod
    def load_from_definition_file(cls, path: str) -> "SKAssistantConfigModel":
        with open(path, 'r') as file:
            yaml_data = yaml.safe_load(file)
            return cls(
                name=yaml_data['name'],
                description=yaml_data['description'],
                instructions=yaml_data['instructions']
            )

In [3]:
from typing import Any, Callable, Optional, Union

from semantic_kernel.sk_pydantic import SKBaseModel
from semantic_kernel.orchestration.sk_function_new import SKFunctionNew as SKFunction
from semantic_kernel.orchestration.native_function import NativeFunction
from semantic_kernel.orchestration.semantic_function import SemanticFunction

class Plugin(SKBaseModel):
    name: str
    description: str = ""
    functions: dict[str, SKFunction] = {}

    def __init__(self, name: str, functions: list["SKFunction"] | None = None):
        if functions:
            functions_dict = {func.name: func for func in functions}
        else:
            functions_dict = {}
        super().__init__(name=name, functions=functions_dict)
        for function in functions:
            function.plugin_name = name

    @classmethod
    def from_class(cls, name: str, class_object: Any) -> "Plugin":
        functions = [
            NativeFunction(getattr(class_object, function))
            for function in dir(class_object)
            if not function.startswith("__")
            and hasattr(getattr(class_object, function), "__sk_function__")
            and getattr(class_object, function).__sk_function__
        ]
        return cls(name, functions)

    def add_function(self, function: SKFunction):
        function.plugin_name = self.name
        self.functions[function.name] = function

    def add_function_from_class(self, class_object: Any):
        functions = [
            getattr(class_object, function)
            for function in dir(class_object)
            if not function.startswith("__")
            and hasattr(function, "__sk_function__")
            and function.__sk_function__
        ]
        for function in functions:
            function.plugin_name = self.name
        self.functions.update(
            {function.name: NativeFunction(function) for function in functions}
        )

    @property
    def fqn_functions(self) -> dict[str, Any]:
        return {func.fully_qualified_name: func for func in self.functions.values()}

ImportError: cannot import name 'SKFunctionNew' from 'semantic_kernel.orchestration.sk_function' (/home/evmattso/workspace/semantic-kernel/python/semantic_kernel/orchestration/sk_function.py)

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai import (
    ChatCompletionClientBase,
    TextCompletionClientBase,
    EmbeddingGeneratorBase,
)

class V1Kernel(Kernel):
    plugins: list["Plugin"] = []
    services: list[ChatCompletionClientBase | TextCompletionClientBase | EmbeddingGeneratorBase] = []

In [ ]:
# openai_helper.py

import openai
import time

from openai.types.beta.threads.run import Run
from openai.types.beta.threads import ThreadMessage
from openai.types.beta.thread import Thread
from openai.pagination import SyncCursorPage

class OpenAIAssistant(OpenAIAssistantBase):
    def __init__(self, model):
        '''
        Initialize the OpenAI client based on the provided model.
        '''
        self.client = openai.OpenAI()
        self.model = model

    def create_assistant(self, name: str, description: str, instructions: str) -> str:
        '''
        Create an assistant with the provided name, description, and instructions.

        Args:
            name (str): The name of the assistant
            description (str): The description of the assistant
            instructions (str): The instructions of the assistant

        Returns:
            str: The ID of the created assistant
        '''
        assistant = self.client.beta.assistants.create(
            name=name,
            instructions=instructions,
            description=description,
            model=self.model,
        )
        return assistant.id
    

    def submit_message(self, assistant_id: str, thread_id: str, user_message: str) -> Run:
        '''
        Submit a message to the provided thread.

        Args:
            assistant_id (str): The ID of the assistant
            thread (str): The thread to submit the message to
            user_message (str): The message to submit

        Returns:
            Run: The run object
        '''
        self.client.beta.threads.messages.create(
            thread_id=thread_id.id, role="user", content=user_message
        )
        return self.client.beta.threads.runs.create(
            thread_id=thread_id.id,
            assistant_id=assistant_id,
        )
    

    def get_response(
            self, 
            thread: Thread
        ) -> SyncCursorPage[ThreadMessage]:
        '''
        Get the response from the provided thread.

        Args:
            thread (Thread): The thread to get the response from

        Returns:
            SyncCursorPage[ThreadMessage]: The response from the thread
        '''
        return self.client.beta.threads.messages.list(thread_id=thread.id, order="asc")


    def create_thread_and_run(self, assistant_id: str, user_input: str):
        '''
        Create a thread and submit a message to it.

        Args:
            assistant_id (str): The ID of the assistant
            user_input (str): The message to submit

        Returns:
            Tuple[Thread, Run]: The thread and run objects
        '''
        thread = self.client.beta.threads.create()
        run = self.submit_message(assistant_id, thread, user_input)
        return thread, run


    def wait_on_run(self, run: Run, thread: Thread):
        '''
        Wait for the run to complete.

        Args:
            run (Run): The run object
            thread (Thread): The thread object

        Returns:
            Run: The updated run object
        '''
        while run.status == "queued" or run.status == "in_progress":
            run = self.client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id,
            )
            time.sleep(0.5)
        return run


    def display_message(self, messages: List[ThreadMessage]) -> None:
        '''
        Display the message from the Thread of messages

        Args:
            messages (List[ThreadMessage]): The messages to pretty print
        '''
        print("# Messages")
        for m in messages:
            print(f"{m.role}: {m.content[0].text.value}")
        print() # separate it with a new line

In [ ]:
# from config_model import SKAssistantConfigModel
# from assistant import Assistant
# from openai_integration import OpenAIHelper

# Load configuration

### uncomment below to run

# definition_str = """
# name: Parrot
# instructions: |
#   Repeat the user message in the voice of a pirate and then end with a parrot sound.
# description: A fun chat bot.
# """
# assistant_config = SKAssistantConfigModel.load_from_definition_string(definition_str)
# assert assistant_config, 'The assistant config cannot be empty'

# # Initialize OpenAI Helper
# oai_model = "gpt-3.5-turbo-1106"
# oai_assistant = OpenAIAssistant(oai_model)
# assistant_id = oai_assistant.create_assistant(
#     name=assistant_config.name,
#     instructions=assistant_config.instructions,
#     description=assistant_config.description,
# )

# # Process messages
# messages = [
#     "Fortune favors the bold.",
#     "I came, I saw, I conquered.",
#     "Practice makes perfect."
# ]

# thread_run_pairs = [oai_assistant.create_thread_and_run(assistant_id, message) for message in messages]

# for thread_id, run in thread_run_pairs:
#     run = oai_assistant.wait_on_run(run, thread_id)
#     oai_assistant.display_message(oai_assistant.get_response(thread_id))

In [ ]:
import os
from os import path

print(os.getcwd())

In [ ]:
# Define an Assistants plugin
assistant = Plugin(
    "Assistant",
    functions=[
        SemanticFunction.from_path(
            path=path.join(
                os.getcwd(),
                "Assistants/ParrotAssistant.yaml"
            )
        )
    ],
)